#Paquetes necesarios

In [4]:
import cv2  
import math 

from ultralytics import YOLO
import pytesseract
import numpy as np

In [11]:
# Tamaño deseado para mostrar las imágenes
desired_size = (400, 300)

# Configuración personalizada para pytesseract
custom_config = r'--oem 3 --psm 6 outputbase alphanumeric'

# Visualización de las imágenes
def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Carga del modelo YOLO
model = YOLO('yolov8n.pt')  # Asegúrate de tener la implementación de YOLO cargada

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Ruta de la imagen a procesar
image_path = "coche2.jpeg"

# Leer la imagen
img = cv2.imread(image_path)

# Realizar la detección de objetos con YOLO
results = model(img, stream=True)

# Variable para indicar si se detectó un coche
car_detected = False

# Para cada detección
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Coordenadas del contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros

        # Clase
        cls = int(box.cls[0])

        # Si la clase detectada es un coche
        if classNames[cls] == "car":
            # Confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confidence --->", confidence)

            print("Class name -->", classNames[cls])

            # Convertir identificador numérico de clase a un color RGB
            escala = int((cls / len(classNames)) * 255 * 3)
            if escala >= 255 * 2:
                R = 255
                G = 255
                B = escala - 255 * 2
            else:
                if escala >= 255:
                    R = 255
                    G = escala - 255
                    B = 0
                else:
                    R = escala
                    G = 0
                    B = 0

            # Dibujar el contenedor y la clase
            cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
            cv2.putText(img, classNames[cls], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

            show_image('Images con clases detectadas', img)

            # Si se detecta un coche, actualiza la variable
            car_detected = True
            # Mostrar solo la región de interés alrededor del coche
            car_roi = img[y1:y2, x1:x2]
            show_image('Región de interés (coche)', car_roi)

# Si se detectó un coche, realiza la detección de matrículas
if car_detected:
    # Convertimos la imagen a escala de grises
    gray = cv2.cvtColor(car_roi, cv2.COLOR_BGR2GRAY)
    show_image('Escala de grises', gray)

    # Aplicamos un threshold
    thresh = cv2.threshold(gray, 140, 255, cv2.THRESH_BINARY_INV)[1]
    show_image('Threshold', thresh)

    # Buscamos los contornos presentes
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    canvas = np.zeros_like(car_roi)
    cv2.drawContours(canvas, contours, -1, (0, 255, 0), 2)
    show_image('Bordes', canvas)

    # Filtrar contornos en la mitad inferior de la imagen
    height, width = car_roi.shape[:2]
    lower_half_contours = [cnt for cnt in contours if cv2.boundingRect(cnt)[1] > height // 2]

    # Ordenar contornos por área (de mayor a menor)
    lower_half_contours = sorted(lower_half_contours, key=cv2.contourArea, reverse=True)

    # Obtener el contorno más grande
    if lower_half_contours:
        largest_contour = lower_half_contours[0]

        # Visualización del contorno más grande
        canvas = np.zeros_like(car_roi)
        cv2.drawContours(canvas, [largest_contour], -1, (0, 255, 0), 2)
        show_image('Contorno más grande en la mitad inferior', canvas)

        # Extraer la región correspondiente al contorno más grande
        x, y, w, h = cv2.boundingRect(largest_contour)
        region_of_interest = car_roi[y:y + h, x:x + w]

        # Mostrar la matrícula
        show_image('Matricula', region_of_interest)

        # Utilizar pytesseract para la detección de texto en la imagen preprocesada
        text = pytesseract.image_to_string(region_of_interest, config=custom_config)
        print("Matrícula detectada:", text)

else:
    print("No se detectó un coche en la imagen.")

Confidence ---> 0.94
Class name --> car


0: 320x640 1 car, 223.5ms
Speed: 4.7ms preprocess, 223.5ms inference, 5.0ms postprocess per image at shape (1, 3, 320, 640)


Matrícula detectada: iKeFA 5627]



Desde cámara, detección con yolov8 y modelo nano. Visualización propia

In [38]:
import numpy as np

# Tamaño deseado para mostrar las imágenes
desired_size = (400, 300)  # Puedes ajustar estos valores según tus necesidades

# Visualización de las imágenes
def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Ruta de la imagen
image_path = "coche2.jpeg"
img = cv2.imread(image_path)

# Convertirmos la imagen a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
show_image('Escala de grises', gray)

# Aplicamos un threshold
thresh = cv2.threshold(gray, 140, 255, cv2.THRESH_BINARY_INV)[1]
show_image('Threshold', thresh)

# Buscamos los contornos presentes
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
canvas = np.zeros_like(img)
cv2.drawContours(canvas, contours, -1, (0, 255, 0), 2)
show_image('Bordes', canvas)

# Filtrar contornos en la mitad inferior de la imagen
height, width = img.shape[:2]
lower_half_contours = [cnt for cnt in contours if cv2.boundingRect(cnt)[1] > height // 2]

# Ordenar contornos por área (de mayor a menor)
lower_half_contours = sorted(lower_half_contours, key=cv2.contourArea, reverse=True)

# Obtener el contorno más grande
if lower_half_contours:
    largest_contour = lower_half_contours[0]

    # Visualización del contorno más grande
    canvas = np.zeros_like(img)
    cv2.drawContours(canvas, [largest_contour], -1, (0, 255, 0), 2)
    show_image('Largest Contour in Lower Half', canvas)

    # Extraer la región correspondiente al contorno más grande
    x, y, w, h = cv2.boundingRect(largest_contour)
    region_of_interest = img[y:y + h, x:x + w]

    # Visualización de la región de interés
    show_image('Region of Interest', region_of_interest)
else:
    print("No se encontraron contornos en la mitad inferior de la imagen.")

Diversos modelos preentrenados, visualizando

In [78]:
# Carga del modelo
#model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



FileNotFoundError: C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4 does not exist

Intregración con seguimiento (tracking)
Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
#model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
#filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
filename = "D:/GH010196s.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [ ]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

Reconocimiento decaracteres tras instalar easyocr

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')